# 2023-08-25 5 - Estimating constraint level.ipynb
Seems like selecting epsilon is important. Can we estimate it based on the statistics of the ERM error? 

## Fetching runs

In [31]:
import wandb
from math import isnan 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wandb
from tqdm import tqdm
# from cycler import cycler
import matplotlib as mpl
api = wandb.Api()
project = "Autoformer"
workspace = "alelab"
experiment_run_name_suffix="-newloss"

runs = api.runs(f"{workspace}/{project}")
all_runs = []
for run in tqdm(runs):
    if run.State=="finished" or True:
        for split in ["train", "test","val"]:
            #for metric in ["mse",]:
            for metric in ["loss",]: #had to change in this plot because I messed up. The 'loss' name is incorrect (it's not the lagrange-added loss.)
                pred_len = run.config["pred_len"]
                metrics = np.zeros(pred_len)
                #if run.sweep.id in ["tkol4t1r","c7xaz5e8"]:
                if run.sweep.id in ["c7xaz5e8"]:
                    for i in range(pred_len):
                        run_dict = {**run.config}
                        #run_dict["constraint_level"] = constraint_level
                        #run_dict[f"{metric}"] = run.summary[f"{metric}/{split}/{i}"]
                        run_dict[f"{metric}"] = run.summary.get(f"{metric}/{split}/{i}",run.summary.get(f"mse/{split}/{i}",np.nan)) #god forgive me for this line
                        run_dict["step"]=i
                        run_dict["split"] = split
                        run_dict["run_id"] = run.id
                        # Get either Constrained/ or ERM/ from the run name, then append model name.
                        #print("run.name", run.name)
                        #debug if ERM run
                        run_dict["Algorithm"] = f"{run.name.split('/')[0]} {run.config['model']}"
                        run_dict["sweep_id"] = run.sweep.id
                        #print("Algorithm", run_dict["Algorithm"])
                        all_runs.append(run_dict)

df = pd.DataFrame(all_runs)

100%|██████████| 240/240 [00:13<00:00, 17.75it/s]


In [5]:
df[['run_id',"sweep_id",'Algorithm','model','constraint_level','pred_len']].drop_duplicates().sort_values(['model',"pred_len","constraint_level"])

,run_id,sweep_id,Algorithm,model,constraint_level,pred_len
5184,tm9558j0,c7xaz5e8,ERM-5e Autoformer,Autoformer,-1,96
4800,419htb1s,c7xaz5e8,ERM-5e Autoformer,Autoformer,-1,192
4128,ulymlsbg,c7xaz5e8,ERM-5e Autoformer,Autoformer,-1,336
2688,hulz8sfh,c7xaz5e8,ERM-5e Autoformer,Autoformer,-1,720
2496,3n6rphzo,c7xaz5e8,ERM-5e DLinear,DLinear,-1,96
2112,wv6uqw5y,c7xaz5e8,ERM-5e DLinear,DLinear,-1,192
1440,tubw60e7,c7xaz5e8,ERM-5e DLinear,DLinear,-1,336
0,ma63fvn6,c7xaz5e8,ERM-5e DLinear,DLinear,-1,720


In [32]:
# Get a single run to calculate stats.
df_af = df[(df["Algorithm"].str.contains("Autoformer")) & (df["run_id"]=='tm9558j0')]

In [41]:
split96=df_af[['run_id','pred_len','loss','step','split']] \
    .query("split=='val'").sort_values(['pred_len']).head(50)

In [42]:
split96

,run_id,pred_len,loss,step,split
7968,tm9558j0,96,0.335980,0,val
8037,tm9558j0,96,0.556818,69,val
8036,tm9558j0,96,0.556359,68,val
8035,tm9558j0,96,0.556551,67,val
8034,tm9558j0,96,0.557422,66,val
8033,tm9558j0,96,0.557279,65,val
8032,tm9558j0,96,0.556386,64,val
8031,tm9558j0,96,0.555836,63,val
8030,tm9558j0,96,0.555406,62,val
8029,tm9558j0,96,0.555372,61,val


In [40]:
split96.loss.describe().reset_index()

,index,loss
0,count,50.000000
1,mean,0.550739
2,std,0.032001
3,min,0.335980
4,25%,0.554996
5,50%,0.556372
6,75%,0.557312
7,max,0.559179


Who knew! the $\epsilon$ that worked best was also the mean and median!

# Get IQR for all splits and lengths:

In [53]:
all_runs = df[(df["Algorithm"].str.contains("Autoformer"))]
stats=all_runs.query("split=='val'").groupby(['pred_len'])['loss'].describe()
stats.transpose()

pred_len,96,192,336,720
count,96.000000,192.000000,336.000000,720.000000
mean,0.525741,0.597614,0.665794,0.806109
std,0.051377,0.086303,0.105595,0.129163
min,0.335980,0.411522,0.470944,0.477714
25%,0.516589,0.553534,0.560144,0.698106
50%,0.553416,0.567745,0.682352,0.862805
75%,0.556407,0.665850,0.741705,0.912515
max,0.559179,0.779780,0.970453,1.008879


In [55]:
stats["50%"]

pred_len
96     0.553416
192    0.567745
336    0.682352
720    0.862805
Name: 50%, dtype: float64